# ETL OF PROJECT

In [202]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Extract data

In [221]:
# Define raw data path
student_path = "../data/raw/2024 UWA Student Data.xlsx"
government_path = "../data/raw/2024 allocation of units of study.xlsx"

# Load datasets
student_df = pd.read_excel(student_path, sheet_name="Sheet1")
government_df = pd.read_excel(government_path, sheet_name="2024AllocationOfUnitsOfStudy")

## Tramsform data

### Exploratory Data Analysis

In [222]:
def explore_dataframe(df, df_name):
    print(f"--- Exploring DataFrame: {df_name} ---")
    print("\nShape:")
    print(df.shape)
    print("\nInfo:")
    df.info()
    print("\nFirst 5 Rows:")
    print(df.head())
    print("\nDescriptive Statistics (for numeric columns):")
    print(df.describe())
    print("\nMissing Values Count:")
    print(df.isnull().sum())
    print("-" * 50 + "\n")

In [223]:
# Extract relevant sheets
explore_dataframe(student_df, "Student Data")

--- Exploring DataFrame: Student Data ---

Shape:
(17672, 11)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17672 entries, 0 to 17671
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   UWACourseID                 17672 non-null  object 
 1   CourseTypeBroadName         17671 non-null  object 
 2   UWAUnitID                   17671 non-null  float64
 3   FundingGroupName            17671 non-null  object 
 4   UnitLevelCode               17671 non-null  float64
 5   UnitLevelName               17671 non-null  object 
 6   UnitPrimaryFOEDetailedName  17671 non-null  object 
 7   UnitPrimaryFOENarrowName    17671 non-null  object 
 8   UnitPrimaryFOECode          17671 non-null  float64
 9   UnitPrimaryFOEBroadName     17671 non-null  object 
 10  2024 EFTSL                  17672 non-null  float64
dtypes: float64(4), object(7)
memory usage: 1.5+ MB

First 5 Rows:
   UWACourseID

In [224]:
# Extract relevant sheets
explore_dataframe(government_df, "Government Data")

--- Exploring DataFrame: Government Data ---

Shape:
(441, 13)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441 entries, 0 to 440
Data columns (total 13 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   Funding Cluster                                                    441 non-null    object
 1   Discipline Code
(FOE)                                              441 non-null    int64 
 2   2024 Maximum Student Contibution                                   441 non-null    int64 
 3   2024 Commonwealth Contribution                                     441 non-null    int64 
 4   2024 Grandfathered Maximum Student Contibution                     441 non-null    int64 
 5   2024 Grandfathered Commonwealth Contribution                       441 non-null    int64 
 6   Funding Cluster varies for FOE depending on E312 or E392 (Ye

### Data Cleaning

### Missing Values Analysis

In [225]:
# For student data, there is one row - grand total row - that has all NaN values. We can remove this row.
student_df.dropna(axis= 0,how='any', inplace=True)
student_df.isna().sum()

UWACourseID                   0
CourseTypeBroadName           0
UWAUnitID                     0
FundingGroupName              0
UnitLevelCode                 0
UnitLevelName                 0
UnitPrimaryFOEDetailedName    0
UnitPrimaryFOENarrowName      0
UnitPrimaryFOECode            0
UnitPrimaryFOEBroadName       0
2024 EFTSL                    0
dtype: int64

### Remove duplicates

In [226]:
# remove duplicates from both datasets
student_df.drop_duplicates(inplace=True)
government_df.drop_duplicates(inplace=True)

#### Normalnize column names

In [227]:
# Student Data columns
# create rename map dictionary
rename_map = {
    'UWACourseID': 'course_id',
    'CourseTypeBroadName': 'course_type_broad',
    'UWAUnitID': 'unit_id',
    'FundingGroupName': 'funding_group',
    'UnitLevelCode': 'unit_level_code',
    'UnitLevelName': 'unit_level_name',
    'UnitPrimaryFOEDetailedName': 'unit_foe_detailed',
    'UnitPrimaryFOENarrowName': 'unit_foe_narrow',
    'UnitPrimaryFOECode': 'foe_code',
    'UnitPrimaryFOEBroadName': 'unit_foe_broad',
    '2024 EFTSL': 'eftsl_2024'
}
# rename columns
student_df.rename(columns=rename_map, inplace=True)

In [228]:
# government Data columns
# create rename map dictionary
rename_map = {
    'Funding Cluster': 'funding_cluster',
    'Discipline Code\n(FOE)': 'foe_code',
    '2024 Maximum Student Contibution': 'max_student_contrib_2024',
    '2024 Commonwealth Contribution': 'commonwealth_contrib_2024',
    '2024 Grandfathered Maximum Student Contibution': 'max_student_contrib_gf_2024',
    '2024 Grandfathered Commonwealth Contribution': 'commonwealth_contrib_gf_2024',
    'Funding Cluster varies for FOE depending on E312 or E392 (Yes/No)': 'is_funding_cluster_variable',
    'Special Course Type Code for the Course of Study\n(E312 of 27)': 'special_course_code',
    'Maximum student contribution indicator\n(E392 =8)': 'max_contrib_indicator',
    'DETAILED Discipline (FOE) - Title': 'foe_detailed_title',
    'DETAILED Discipline (FOE) ': 'foe_detailed',
    'NARROW Discipline (FOE) ': 'foe_narrow',
    'BROAD Discipline (FOE) ': 'foe_broad'
}
government_df.rename(columns=rename_map, inplace=True)

#### Normalnize column values

Remove leading/trailing spaces from string columns

In [229]:
student_df = student_df.map(lambda x: x.strip() if isinstance(x, str) else x)
government_df = government_df.map(lambda x: x.strip() if isinstance(x, str) else x)

normalize boolean column values to yes/no

In [230]:
# check the data
government_df[['is_funding_cluster_variable', 'special_course_code', 'max_contrib_indicator']].head()

,is_funding_cluster_variable,special_course_code,max_contrib_indicator
0,Yes,Not E312=27,Not E392=8
1,Yes,27,Not E392=8
2,Yes,Not E312=27,8
3,Yes,Any E312 value,Not E392=8
4,Yes,Any E312 value,8


i) FOE 090701 (psychology) units of study are classified under funding cluster 1, unless the course's "special course type code" is flagged as being a postgraduate clinical psychology course (E312=27, "A course of study in clinical psychology (as defined in the Commonwealth Grant Scheme Guidelines"). That is,  if the FOE is 090701 and E312 =27 (**postgraduate clinical psychology**) then the unit of study is allocated to **funding cluster 2**, otherwise it is **allocated to funding cluster 1**.

In [231]:
# check the data
government_df['special_course_code'].value_counts()

special_course_code
Any E312 value    438
Not E312=27         2
27                  1
Name: count, dtype: int64

In [232]:
# transform to 'Yes'/'No'
government_df['special_course_code'] = np.where(
    government_df['special_course_code'] == 27, 
    'Yes', 
    'No'
)
government_df['special_course_code'].value_counts()

special_course_code
No     440
Yes      1
Name: count, dtype: int64

ii)  Professional pathway psychology and professional pathway social work units of study are classified under funding cluster 2. They are identified as having a narrow FOE of 0907 (behavioural Science) or a narrow FOE of 0905 (Human Welfare Studies and Services) respectively and having an E392 (Maximum student contribution indicator) equal to 8 (**students who are not subject to pre-2021 grandfathering but are doing a professional pathways units**). 
Note: In the "2024AllocationOfUnitsOfStudy" tab, the grandfathered amounts for pathway FOEs are "greyed out" (columns E and F). This is because the E392 = "8" value only applied to non-grandfathered students. The value is still shown for theoretical reference. However, grandfathering arrangements for units now defined as professional pathway students are the same as those for non-professional pathway students.

In [233]:
government_df['max_contrib_indicator'].value_counts()

max_contrib_indicator
Any E392 value    414
Not E392=8         14
8                  13
Name: count, dtype: int64

In [234]:
# transform to 'Yes'/'No'
government_df['max_contrib_indicator'] = np.where(
    government_df['max_contrib_indicator'] == 8, 
    'Yes', 
    'No'
)
government_df['max_contrib_indicator'].value_counts()

max_contrib_indicator
No     428
Yes     13
Name: count, dtype: int64

Split the grouped-values

In [235]:
# check the data
student_df['funding_group']

1        Domestic - C'wealth Supported
2        Domestic - C'wealth Supported
3        Domestic - C'wealth Supported
4        Domestic - C'wealth Supported
5        Domestic - C'wealth Supported
                     ...              
17667            Domestic - Fee-Paying
17668            Domestic - Fee-Paying
17669          International - Onshore
17670          International - Onshore
17671            Domestic - Fee-Paying
Name: funding_group, Length: 17671, dtype: object

In [236]:
# split funding_group into funding_type and funding_nationality
split_data = student_df['funding_group'].str.split(' - ', expand=True)
student_df['funding_nation'] = split_data[0]
student_df['funding_type'] = split_data[1]
# drop original funding_group column
student_df.drop(columns=['funding_group'], inplace=True)
# check the data
student_df[['funding_nation', 'funding_type']].head()

,funding_nation,funding_type
1,Domestic,C'wealth Supported
2,Domestic,C'wealth Supported
3,Domestic,C'wealth Supported
4,Domestic,C'wealth Supported
5,Domestic,C'wealth Supported


In [237]:
# check the data
government_df[['foe_detailed', 'foe_narrow', 'foe_broad']]

,foe_detailed,foe_narrow,foe_broad
0,090701 - Psychology,0907 - Behavioural Science,09 - SOCIETY AND CULTURE
1,090701 - Psychology,0907 - Behavioural Science,09 - SOCIETY AND CULTURE
2,090701 - Psychology,0907 - Behavioural Science,09 - SOCIETY AND CULTURE
3,090700 - Behavioural Science,0907 - Behavioural Science,09 - SOCIETY AND CULTURE
4,090700 - Behavioural Science,0907 - Behavioural Science,09 - SOCIETY AND CULTURE
...,...,...,...
436,"060799 - Dental Studies, n.e.c.",0607 - Dental Studies,06 - HEALTH
437,061100 - Veterinary Studies,0611 - Veterinary Studies,06 - HEALTH
438,061101 - Veterinary Science,0611 - Veterinary Studies,06 - HEALTH
439,061103 - Veterinary Assisting,0611 - Veterinary Studies,06 - HEALTH


In [238]:
# split the data
columns_to_clean = ['foe_detailed', 'foe_narrow', 'foe_broad']
for column in columns_to_clean:
    government_df[column] = government_df[column].str.split(' - ', n=1).str[1]

# check the data
government_df[['foe_detailed', 'foe_narrow', 'foe_broad']].head()

,foe_detailed,foe_narrow,foe_broad
0,Psychology,Behavioural Science,SOCIETY AND CULTURE
1,Psychology,Behavioural Science,SOCIETY AND CULTURE
2,Psychology,Behavioural Science,SOCIETY AND CULTURE
3,Behavioural Science,Behavioural Science,SOCIETY AND CULTURE
4,Behavioural Science,Behavioural Science,SOCIETY AND CULTURE


Transform data types from float to int

In [239]:
# student data
columns_to_int= ['course_id', 'unit_id', 'unit_level_code', 'foe_code']
for col in columns_to_int:
    student_df[col] = student_df[col].astype('int64')

### Data integration

### **compare foe_code in both datasets**

In [240]:
# extract unique foe_codes from both datasets
student_codes = set(student_df['foe_code'].unique())
gov_codes = set(government_df['foe_code'].unique())

#check if all student codes are in government codes
all_in = student_codes.issubset(gov_codes)
print(all_in)   # returns False or True

False


In [241]:
# find out which codes in student_codes are not in gov_codes
diff = student_codes - gov_codes
print(diff)

{60000, 120000}


In [242]:
# find out which codes in gov_codes are not in student_codes
mask = student_df['foe_code'].astype(str).str.contains("60000|120000", na=False)
student_df[mask].head()

,course_id,course_type_broad,unit_id,unit_level_code,unit_level_name,unit_foe_detailed,unit_foe_narrow,foe_code,unit_foe_broad,eftsl_2024,funding_nation,funding_type
4270,4638,Higher Degree Research,242719,5,Level Five,Health,Health,60000,Health,1.000,Domestic,RTP
14344,3057,Postgraduate Coursework,192415,5,Level Five,Mixed Field Programmes,Mixed Field Programmes,120000,Mixed Field Programmes,0.125,International,Onshore Exchange
14354,3057,Postgraduate Coursework,205484,5,Level Five,Mixed Field Programmes,Mixed Field Programmes,120000,Mixed Field Programmes,0.125,International,Onshore Exchange


**The FOE codes 60000 and 120000 appear only in the student dataset, which needs to be clarified further.**

In [ ]:
# merge datasets on FOE Broad
# choose left join to keep all student records
merged_df = pd.merge(student_df, government_df, on = 'foe_code', how ='left')

In [244]:
counts = government_df['foe_code'].value_counts()
counts_greater_than_one = counts[counts > 1]
print(counts_greater_than_one)

foe_code
90701    3
90507    2
90700    2
90515    2
90513    2
90511    2
90509    2
90599    2
90505    2
90503    2
90501    2
90500    2
90799    2
Name: count, dtype: int64


**Need to map the special code to the student data and update the calculation.**

In [ ]:
# Create new column to calculate the total funding for domestic students
# calculation needs to be updated on specical code and international students payment        
merged_df['stud_payment'] = 0.0
merged_df['gov_payment'] = 0.0

for row in merged_df.itertuples():
    if row.funding_nation == 'Domestic':
        if row.max_student_contrib_2024 <= row.max_student_contrib_gf_2024:
            merged_df.loc[row.Index, 'stud_payment'] = row.max_student_contrib_2024 * row.eftsl_2024
        else:
            merged_df.loc[row.Index, 'stud_payment'] = row.max_student_contrib_gf_2024 * row.eftsl_2024
    else:
        merged_df.loc[row.Index, 'stud_payment'] = row.eftsl_2024 * 20000 # need student payment for international students
  
for row in merged_df.itertuples():
    if row.funding_nation == 'Domestic':
        if row.commonwealth_contrib_2024 >= row.commonwealth_contrib_gf_2024:
            merged_df.loc[row.Index, 'gov_payment'] = row.commonwealth_contrib_2024 * row.eftsl_2024
        else:
            merged_df.loc[row.Index, 'gov_payment'] = row.commonwealth_contrib_gf_2024 * row.eftsl_2024
    else:
        merged_df.loc[row.Index, 'gov_payment'] = 0.0 # no government payment for international students

merged_df['total_payment'] = merged_df['stud_payment'] + merged_df['gov_payment']

In [246]:
# export merged data
merged_df.to_csv("../data/processed/merged_data.csv", index=False)

In [247]:
merged_df.head()

,course_id,course_type_broad,unit_id,unit_level_code,unit_level_name,unit_foe_detailed,unit_foe_narrow,foe_code,unit_foe_broad,eftsl_2024,...,is_funding_cluster_variable,special_course_code,max_contrib_indicator,foe_detailed_title,foe_detailed,foe_narrow,foe_broad,stud_payment,gov_payment,total_payment
0,4951,Postgraduate Coursework,240072,3,Level Three,Medical Studies,Medical Studies,60100,Health,97.0,...,No,No,No,Medical Studies,Medical Studies,Medical Studies,HEALTH,1233840.0,2948315.0,4182155.0
1,4951,Postgraduate Coursework,240586,3,Level Three,Medical Studies,Medical Studies,60100,Health,92.5,...,No,No,No,Medical Studies,Medical Studies,Medical Studies,HEALTH,1176600.0,2811537.5,3988137.5
2,4951,Postgraduate Coursework,239439,4,Level Four,Medical Studies,Medical Studies,60100,Health,86.5,...,No,No,No,Medical Studies,Medical Studies,Medical Studies,HEALTH,1100280.0,2629167.5,3729447.5
3,4951,Postgraduate Coursework,240587,4,Level Four,Medical Studies,Medical Studies,60100,Health,86.5,...,No,No,No,Medical Studies,Medical Studies,Medical Studies,HEALTH,1100280.0,2629167.5,3729447.5
4,4733,Undergraduate,243526,1,Level One,Statistics,Mathematical Sciences,10103,Natural And Physical Sciences,80.0,...,No,No,No,Statistics,Statistics,Mathematical Sciences,NATURAL AND PHYSICAL SCIENCES,355600.0,1193200.0,1548800.0


### Dimension Tables

#### Dimension Table for Course category

Include the 'course_type_broad', 'unit_id', 'unit_level_code', 'unit_level_name'.

In [288]:
dim_course = merged_df[['course_type_broad', 'unit_id','unit_level_code','unit_level_name']].drop_duplicates().reset_index(drop=True)
dim_course.insert(0, 'course_info_id', dim_course.index + 1 )
dim_course.head()

,course_info_id,course_type_broad,unit_id,unit_level_code,unit_level_name
0,1,Postgraduate Coursework,240072,3,Level Three
1,2,Postgraduate Coursework,240586,3,Level Three
2,3,Postgraduate Coursework,239439,4,Level Four
3,4,Postgraduate Coursework,240587,4,Level Four
4,5,Undergraduate,243526,1,Level One


#### Dimension Table for Funding category

In [269]:
dim_funding = merged_df[['funding_cluster','funding_nation','funding_type']].drop_duplicates().reset_index(drop=True)
dim_funding.insert(0, 'funding_id', dim_funding.index + 1)
dim_funding.head()

,funding_id,funding_cluster,funding_nation,funding_type
0,1,Funding Cluster 4,Domestic,C'wealth Supported
1,2,Funding Cluster 2,Domestic,C'wealth Supported
2,3,Funding Cluster 1,Domestic,C'wealth Supported
3,4,Funding Cluster 2,International,Onshore
4,5,Funding Cluster 4,Domestic,RTP


#### Dimension table for FOE category

In [300]:
dim_foe = merged_df[['foe_detailed', 'foe_narrow', 'foe_broad', 'is_funding_cluster_variable','special_course_code', 'max_contrib_indicator']].drop_duplicates().reset_index(drop=True)
dim_foe.insert(0, 'foe_id', dim_foe.index + 1)
dim_foe.head()

,foe_id,foe_detailed,foe_narrow,foe_broad,is_funding_cluster_variable,special_course_code,max_contrib_indicator
0,1,Medical Studies,Medical Studies,HEALTH,No,No,No
1,2,Statistics,Mathematical Sciences,NATURAL AND PHYSICAL SCIENCES,No,No,No
2,3,Economics,Economics and Econometrics,SOCIETY AND CULTURE,No,No,No
3,4,Accounting,Accounting,MANAGEMENT AND COMMERCE,No,No,No
4,5,Organisation Management,Business and Management,MANAGEMENT AND COMMERCE,No,No,No


#### Fact table

In [298]:
fact_table = merged_df.copy()

In [302]:
# merge with course table
fact_table = pd.merge(fact_table, dim_course, on=['course_type_broad', 'unit_id','unit_level_code','unit_level_name'], how='left')

# merge with funding table
fact_table = pd.merge(fact_table, dim_funding, on=['funding_cluster','funding_nation','funding_type'], how='left')

# merge with foe table
fact_table = pd.merge(fact_table, dim_foe, on=['foe_detailed', 'foe_narrow', 'foe_broad', 'is_funding_cluster_variable','special_course_code', 'max_contrib_indicator'], how='left')

In [305]:
# select relevant columns for fact table
fact_table = fact_table[['course_id', 'course_info_id', 'funding_id', 'foe_code', 'foe_id', 'eftsl_2024','max_student_contrib_2024', 'commonwealth_contrib_2024', 'max_student_contrib_gf_2024',
'commonwealth_contrib_gf_2024', 'stud_payment', 'gov_payment', 'total_payment']]
fact_table.head()

,course_id,course_info_id,funding_id,foe_code,foe_id,eftsl_2024,max_student_contrib_2024,commonwealth_contrib_2024,max_student_contrib_gf_2024,commonwealth_contrib_gf_2024,stud_payment,gov_payment,total_payment
0,4951,1,1,60100,1,97.0,12720.0,30395.0,12720.0,30395.0,1233840.0,2948315.0,4182155.0
1,4951,2,1,60100,1,92.5,12720.0,30395.0,12720.0,30395.0,1176600.0,2811537.5,3988137.5
2,4951,3,1,60100,1,86.5,12720.0,30395.0,12720.0,30395.0,1100280.0,2629167.5,3729447.5
3,4951,4,1,60100,1,86.5,12720.0,30395.0,12720.0,30395.0,1100280.0,2629167.5,3729447.5
4,4733,5,2,10103,2,80.0,4445.0,14915.0,4445.0,14915.0,355600.0,1193200.0,1548800.0


## Load data

In [306]:
# set processed data path
PROCESSED_DATA_PATH = '../data/processed/'

# save dimension tables and fact table as csv files
dim_course.to_csv(os.path.join(PROCESSED_DATA_PATH, 'dim_course.csv'), index=False)
dim_funding.to_csv(os.path.join(PROCESSED_DATA_PATH, 'dim_funding.csv'), index=False)
dim_foe.to_csv(os.path.join(PROCESSED_DATA_PATH, 'dim_foe.csv'), index=False)
fact_table.to_csv(os.path.join(PROCESSED_DATA_PATH, 'fact_table.csv'), index=False)